In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from tqdm import tqdm
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
def count_tfidf_without_stop_removal(train,create_test=True):
    print("Original Train File: ", train.shape)
    train = train.drop(train[train['user_review'].isna()].index) #Dropping na index from text
    empty_idx = train[train['user_review'] == ' '].index
    train = train.drop(index=empty_idx)
    print("Train File After Dropping na and ' ' empty string: ",train.shape)
    user_review = train.user_review.values
    idx = []
    for i,ur in enumerate(user_review): #Checking whether there are still values which are not string (nan is a float)
        if not isinstance(ur,str):
            idx.append(i)
    train = train.drop(index=idx)
    print("Train File After Double Check of na and ' ' empty string: ",train.shape)
    user_review = train.user_review.values #Safe since we have updated train at each step
    docs = nlp.pipe(user_review) #fitting it to the pipeline
    cleaned_text = []
    pbar = tqdm(total=len(user_review))
    for i,doc in enumerate(docs):
        doc_vec = []
        for token in doc:
            doc_vec.append(token.text)
        cleaned_text.append(doc_vec)
        if i%1000 == 0  and i != 0:
            pbar.update(1000)
    pbar.close()
    print("++++++++++ CLEANING DONE +++++++++++++")
    print("+++++++++ ORIGINAL REVIEW +++++++++++")
    print(user_review[0])
    print("+++++++++ TEXT CLEANED ++++++++++++++")
    print(" ".join(cleaned_text[0]))
    corpus = np.array([" ".join(t) for t in cleaned_text])
    if create_test:
        X_train,X_test,y_train,y_test = train_test_split(corpus,train.user_suggestion.values,stratify=train.user_suggestion.values,test_size=0.2)
        return X_train,X_test,y_train,y_test,corpus
    else:
        return corpus,train.user_suggestion.values,corpus

In [5]:
def test_cleaning_without_stop_removal(test):
    nan_idx = test[test['user_review'].isna()].index
    test.loc[nan_idx,'user_review'] = test.loc[nan_idx,'title']
    test_user_review = test.user_review
    for i,ur in enumerate(test_user_review):
        if not isinstance(ur,str):
            print(i)
    test_data = []
    test_docs = nlp.pipe(test_user_review)
    pbar = tqdm(total=len(test_user_review))
    for i,doc in enumerate(test_docs):
        doc_vec = []
        for token in doc:
            doc_vec.append(token.text)
        test_data.append(doc_vec)
        if i%1000 == 0  and i != 0:
            pbar.update(1000)
    pbar.close()
    test_corpus = np.array([" ".join(t) for t in test_data])
    return test_corpus,test_corpus

In [6]:
train = pd.read_csv("proper_cleaned_data/clean_review_tags_overview_notremoved_stopwords_frequent_nonfrequent_words.csv")
X_train,X_test,y_train,y_test,train_corpus = count_tfidf_without_stop_removal(train)

  0%|          | 0/17472 [00:00<?, ?it/s]

Original Train File:  (17494, 9)
Train File After Dropping na and ' ' empty string:  (17472, 9)
Train File After Double Check of na and ' ' empty string:  (17472, 9)


 97%|█████████▋| 17000/17472 [04:49<00:08, 58.72it/s]


++++++++++ CLEANING DONE +++++++++++++
+++++++++ ORIGINAL REVIEW +++++++++++
i am scared and hearing creepy voices so i will pause for a moment and write a review while i wait for my heart beat to return to atleast somewhat calmer times this game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood but more bubble and clean hello 1990s what charactes there are that isnot trying to kill me were likable and a bit odd i did do a few noob things though such as oh look a class room full of ghosts from dead children lets shine my flashlight on them and stand there staring at them or hmm creepy music i will turn around and see if i can see what is chasing me never before in a game have i been this afraid of finding a locked door
+++++++++ TEXT CLEANED ++++++++++++++
i am scared and hearing creepy voices so i will pause for a moment and write a review while i wait for my heart beat to return to atleast somewhat calmer times this game is adorable and cre

In [ ]:
cv = CountVectorizer(ngram_range=(1,2))
full_corpus = cv.fit_transform(train_corpus)
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)
tf = TfidfTransformer()
tf.fit(full_corpus)
X_train_cv = tf.transform(X_train_cv).toarray()
X_test_cv = tf.transform(X_test_cv).toarray()